In [ ]:
import numpy as np
import mediapipe as mp
import cv2
from collections import deque
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle
def calculate_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)
# Draw skeletons
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
# Start detecting
# Initialize deque (double-ended queue) for tracking wrist positions
num_frames_to_analyze = 30  # Number of frames to consider for movement analysis
left_wrist_positions = deque(maxlen=num_frames_to_analyze)
right_wrist_positions = deque(maxlen=num_frames_to_analyze)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        continue
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
         # Get wrist positions
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        # Update wrist positions history
        left_wrist_positions.append(left_wrist)
        right_wrist_positions.append(right_wrist)
        # Define relevant landmarks for conditions
        shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, 
                landmarks[mp_pose.PoseLandmark.NOSE.value].y]
        # Check for hunchback
        shoulder_hip_angle = calculate_angle(nose, shoulder_left, hip_left)
        if shoulder_hip_angle < 145 or shoulder_hip_angle > 170:  # Adjust this angle based on testing
            cv2.putText(frame, 'Straighten your back and keep your head up', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        # Check for feet lifting
        if knee_left[1] - hip_left[1] > 0.05 and knee_right[1] - hip_right[1] > 0.05:  # Adjust this threshold based on testing
            cv2.putText(frame, 'Lift your feet higher', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        # Check for arm swing
        if len(left_wrist_positions) == num_frames_to_analyze:
            # Calculate movement for left and right wrists
            left_movement = sum([calculate_distance(left_wrist_positions[i], left_wrist_positions[i-1]) for i in range(1, num_frames_to_analyze)])
            right_movement = sum([calculate_distance(right_wrist_positions[i], right_wrist_positions[i-1]) for i in range(1, num_frames_to_analyze)])
            # Define thresholds for movement (adjust these based on testing)
            min_movement_threshold = 0.1
            max_movement_threshold = 0.5
            # Check if the movement is within the expected range
            if left_movement < min_movement_threshold or right_movement < min_movement_threshold:
                cv2.putText(frame, 'Swing your arms more', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            elif left_movement > max_movement_threshold or right_movement > max_movement_threshold:
                cv2.putText(frame, 'Reduce arm swing', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('Slow Joggin Pose Correction', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

2024-04-16 15:55:50.781991: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
